<a href="https://colab.research.google.com/github/kurehajime/openapi_sandbox/blob/main/hatebu_filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# はてなブックマークの人気エントリのうちポジティブなものだけ抽出する

まずはOpen AIのAPIキーを入力してください。

In [76]:
from getpass import getpass
secret = getpass('Enter the secret value: ')

Enter the secret value: ··········


In [65]:
!pip install feedparser
!pip install openai 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai: filename=openai-0.26.5-py3-none-any.whl size=67620 sha256=68593544a6eb8687949b9c5dc2c6ba22ea7ebac74083ce86b6fb2b32c83ebb02
  Stored in directory: /root/.cache/pip/wheels/a7/47/99/8273a59fbd59c303e8ff175416d5c1c9c03a2e83ebf7525a99
Successfully built openai


In [66]:
import feedparser
url = "https://b.hatena.ne.jp/hotentry.rss"

In [58]:
# フィードの取得
feed = feedparser.parse(url)

{'bozo': False, 'entries': [{'id': 'https://anond.hatelabo.jp/20230222194136', 'title': 'ゲームさんぽの次に見るべきYouTubeの教養コンテンツ', 'title_detail': {'type': 'text/plain', 'language': None, 'base': 'https://b.hatena.ne.jp/hotentry.rss', 'value': 'ゲームさんぽの次に見るべきYouTubeの教養コンテンツ'}, 'links': [{'rel': 'alternate', 'type': 'text/html', 'href': 'https://anond.hatelabo.jp/20230222194136'}], 'link': 'https://anond.hatelabo.jp/20230222194136', 'summary': 'ゲームさんぽが終わってしまった。いや正確にはまだ終わったわけじゃないけど、自分がYouTubeに齧り付くきっかけとなったチャンネルなので、どこか名残惜しい。 ゲームさんぽを知って、こんな「教養のお裾分け」をしてくれるものが転がっているという事に驚き、それからYouTubeで似たようなチャンネルを探し回っていた。...', 'summary_detail': {'type': 'text/html', 'language': None, 'base': 'https://b.hatena.ne.jp/hotentry.rss', 'value': 'ゲームさんぽが終わってしまった。いや正確にはまだ終わったわけじゃないけど、自分がYouTubeに齧り付くきっかけとなったチャンネルなので、どこか名残惜しい。 ゲームさんぽを知って、こんな「教養のお裾分け」をしてくれるものが転がっているという事に驚き、それからYouTubeで似たようなチャンネルを探し回っていた。...'}, 'updated': '2023-02-22T10:55:34Z', 'updated_parsed': time.struct_time(tm_year=2023, tm_mon=2, tm_mday=22, tm_hour=10, tm_min

In [142]:
def to_obj(entry):
  title = entry['title']
  link = entry['link']
  description = entry['description']
  return {'title':title,'link':link}

entries = list(map(to_obj,feed['entries']))[:15]

entries_text = '\n'.join([f"[{entry['title']}]({entry['link']})" for entry in entries])

In [147]:
import openai
openai.api_key = secret
completion = openai.Completion.create(
    model="text-davinci-003",
    prompt=f"""From the following news, extract only five rows that contain positive news.

{entries_text}
    """,
    max_tokens=700,
    temperature=0.5)

In [148]:
for comp in completion.choices:
  print(comp.text)


1. [「頭のいい人」が自然とやっている4つのこと。“素直に○○する” ことも意外と大切だった - STUDY HACKER（スタディーハッカー）｜社会人の勉強法＆英語学習](https://studyhacker.net/sharp-people)
2. [家計管理アプリ「マネーフォワード ME」に課金したら、お金に無頓着な私でも節約できるようになった | ライフハッカー・ジャパン](https://www.lifehacker.jp/article/2302-tinyhack-money-forward-me-premium/)
3. [生産性を高めれば高めるほどますます忙しくなる /「限りある時間の使い方」を読んだ - kakakakakku blog](https://kakakakakku.hatenablog.com/entry/2023/02/22/084137)
4. [トマト1個、もやし1袋入り「フライパン炊き込みご飯」にんにく、チーズ、コンソメで簡単、激ウマに炊けます【筋肉料理人】 - メシ通 | ホットペッパーグルメ](https://www.hotpepper.jp/mesitsu/entry/kinniku/2023-00766)
5. [アップルウオッチで血糖値測定する技術、開発が飛躍的進展－関係者](https://www.bloomberg.co.jp/news/articles/2023-02-22/RQHT1BT1UM0W01)
